# Hidden Markov Models with DMX-Learn

## Overview

This notebook provides a comprehensive walkthrough of **Hidden Markov Models (HMMs)** using the `dmx-learn` library. You'll learn what HMMs are, why they're useful, and how to implement them effectively.

## What are Hidden Markov Models?

Hidden Markov Models are powerful probabilistic models for sequential data where:
- The system being modeled is assumed to be a **Markov process** with hidden (unobserved) states
- We observe **emissions** that depend on these hidden states
- The goal is to infer the hidden state sequence and learn the model parameters from observed data

## Why Use HMMs?

HMMs are particularly useful for:
- **Time series analysis**: Modeling sequential data with temporal dependencies
- **Speech recognition**: Identifying phonemes and words from audio signals
- **Bioinformatics**: Gene prediction, protein sequence analysis, and genome annotation
- **Natural language processing**: Part-of-speech tagging and named entity recognition
- **Financial modeling**: Regime detection in market conditions
- **Activity recognition**: Inferring hidden states from sensor data

## What You'll Learn

In this notebook, we'll cover:
1. **Basic HMM concepts**: States, transitions, and emissions
2. **Creating HMMs in dmx-learn**: Setting up model structure and parameters
3. **Training HMMs**: Learning model parameters from data using Expectation-Maximization (EM)
4. **Inference**: Decoding hidden state sequences and computing likelihoods
5. **Estimation tricks**: Key'ing distributions and flattening parameter estimates.

Let's get started!

In [ ]:
import numpy as np
import os 
import re
import codecs
from dmx.stats import *
from dmx.utils.estimation import optimize, best_of
from dmx.utils.optsutil import count_by_value
DATA_LOC = "../data"

## Understanding Hidden Markov Models: Mathematical Framework

### Model Components

A Hidden Markov Model consists of:

1. **Hidden States** $S = \{s_1, s_2, ..., s_N\}$: The unobserved states of the system
2. **Observations** $O = \{o_1, o_2, ..., o_M\}$: The observable outputs/emissions
3. **Initial State Distribution** $\pi$: Probability of starting in each hidden state
   - $\pi_i = P(z_1 = s_i)$
4. **Transition Probabilities** $A$: Probability of moving from one hidden state to another
   - $a_{ij} = P(z_{t+1} = s_j \mid z_t = s_i)$
5. **Emission Probabilities** $B$: Probability of observing output given hidden state
   - $b_j(o_k) = P(x_t = o_k \mid z_t = s_j)$
6. **Length Distribution**: Probability of observing output sequence of a given length $T$
   - $P(T=t)$

### The Likelihood Function

Given a sequence of observations $X = (x_1, x_2, ..., x_T)$ and a hidden state sequence $Z = (z_1, z_2, ..., z_T)$, the joint probability is:

$$P(X, Z \mid \theta) = P(z_1) \prod_{t=1}^{T-1} P(z_{t+1} \mid z_t) \prod_{t=1}^{T} P(x_t \mid z_t) P(T)$$

Where $\theta = \{\pi, A, B\}$ represents all model parameters.

The likelihood of observing $X$ (marginalizing over all possible hidden state sequences) is:

$$P(X \mid \theta) = \sum_{Z} P(X, Z \mid \theta) = \sum_{Z} \pi_{z_1} \prod_{t=1}^{T-1} a_{z_t, z_{t+1}} \prod_{t=1}^{T} b_{z_t}(x_t)P(T)$$

### The `HiddenMarkovModelDistribution` in `dmx-learn`
Below we will call the `help()` function on `HiddenMarkovModelDistribution` to see what parameters are required to define an HMM. 

In [2]:
help(HiddenMarkovModelDistribution)

Help on class HiddenMarkovModelDistribution in module dmx.stats.hidden_markov:

class HiddenMarkovModelDistribution(dmx.stats.pdist.SequenceEncodableProbabilityDistribution)
 |  HiddenMarkovModelDistribution(
 |      topics: Sequence[dmx.stats.pdist.SequenceEncodableProbabilityDistribution],
 |      w: Union[Sequence[float], numpy.ndarray],
 |      transitions: Union[List[List[float]], numpy.ndarray],
 |      taus: Union[List[List[float]], numpy.ndarray, NoneType] = None,
 |      len_dist: Optional[dmx.stats.pdist.SequenceEncodableProbabilityDistribution] = NullDistribution(name=None),
 |      name: Optional[str] = None,
 |      keys: Optional[Tuple[Optional[str], Optional[str], Optional[str]]] = (None, None, None),
 |      terminal_values: Optional[Set[~T]] = None,
 |      use_numba: bool = False
 |  ) -> None
 |
 |  HiddenMarkovModelDistribution object defining HMM compatible with data type T.
 |
 |  Defines an HMM with emission distributions in 'topics' (all must have the same data 

### Mapping Mathematical Parameters to `HiddenMarkovModelDistribution`

Now let's understand how the mathematical components of an HMM map to the parameters in `dmx-learn`:

| Mathematical Component | Parameter Name | Description | Example |
|------------------------|----------------|-------------|---------|
| **Initial Distribution** $\pi$ | `w` | Probability distribution over initial hidden states | `[0.6, 0.4]` for sunny/rainy |
| **Transition Probabilities** $A$ | `transitions` | Distribution modeling state-to-state transitions | Markov chain or transition matrix |
| **Emission Probabilities** $B$ | `topics` | Distributions from `dmx.stats` that model observations given hidden states | List of `CategoricalDistribution` objects |
| **Length Distribution** | `len_dist` | Distribution from `dmx.stats` to model the length of obeserved sequences | `PossionDistribution(lam=5.0)`|

**Key Insights:**

- **`w` (weights)**: Represents $\pi$, the initial state distribution
  - Vector of probabilities, one for each hidden state
  - Must sum to 1.0
  
- **`transitions`**: Represents $A$, the transition model
  - Rows sum to 1.0
  - Defines how the hidden state evolves over time
  - Each row represents transition probabilities from one state
  
- **`topics`**: Represents $B$, the emission/observation model
  - List of probability distributions (one per hidden state)
  - Each distribution models what observations are likely given that hidden state
  - Can be any distribution from `dmx.stats` (Categorical, Gaussian, etc.)

- **`len_dist`**: Models the length of the sequences.
    - Ignored if not passed. 
    - Must be passed to generate (sample) sequences.


### Toy Example: Weather and Activities

Let's consider a simple example to make this concrete:

**Scenario**: You're observing someone's daily activities for a week at a time, and you want to infer the weather (which you cannot directly observe).

- **Hidden States**: Weather conditions
  - $S = \{\text{Sunny}, \text{Rainy}\}$
  
- **Observations**: Activities performed
  - $O = \{\text{walk}, \text{shop}, \text{clean}\}$

**Model Parameters**:

*Initial Distribution* $\pi$:
- $P(z_1 = \text{Sunny}) = 0.6$
- $P(z_1 = \text{Rainy}) = 0.4$

*Transition Probabilities* $A$:
$$A = \begin{bmatrix} 
P(\text{Sunny} \mid \text{Sunny}) & P(\text{Rainy} \mid \text{Sunny}) \\
P(\text{Sunny} \mid \text{Rainy}) & P(\text{Rainy} \mid \text{Rainy})
\end{bmatrix} = \begin{bmatrix} 
0.7 & 0.3 \\
0.4 & 0.6
\end{bmatrix}$$

*Emission Probabilities* $B$:
$$B = \begin{bmatrix}
P(\text{walk} \mid \text{Sunny}) & P(\text{shop} \mid \text{Sunny}) & P(\text{clean} \mid \text{Sunny}) \\
P(\text{walk} \mid \text{Rainy}) & P(\text{shop} \mid \text{Rainy}) & P(\text{clean} \mid \text{Rainy})
\end{bmatrix} = \begin{bmatrix}
0.6 & 0.3 & 0.1 \\
0.1 & 0.4 & 0.5
\end{bmatrix}$$

**Interpretation**:
- On sunny days, people are more likely to walk (0.6) than clean (0.1)
- On rainy days, people are more likely to clean (0.5) than walk (0.1)
- Sunny weather tends to persist (0.7 probability of staying sunny)
- Rainy weather is somewhat persistent (0.6 probability of staying rainy)

**Example Sequence**:
If we observe the sequence: $X = (\text{walk}, \text{shop}, \text{clean})$

We can compute the likelihood for any hidden state sequence. For example, if $Z = (\text{Sunny}, \text{Sunny}, \text{Rainy})$:

$$P(X, Z) = P(\text{Sunny}) \times P(\text{walk} \mid \text{Sunny}) \times P(\text{Sunny} \mid \text{Sunny}) \times P(\text{shop} \mid \text{Sunny}) \times P(\text{Rainy} \mid \text{Sunny}) \times P(\text{clean} \mid \text{Rainy})$$

$$= 0.6 \times 0.6 \times 0.7 \times 0.3 \times 0.3 \times 0.5 = 0.01134$$

The total likelihood $P(X)$ would sum this over all $2^3 = 8$ possible hidden state sequences.

### Defining the `HiddenMarkovModelDistribution`
We can define the `HiddenMarkovModelDistribution` in `dmx-learn` as follows:

In [6]:
# initial states for sunny and rainy
w = np.array([0.6, 0.4])
# define the transition matrix
tpm = [[0.7, 0.3], [0.4, 0.6]]

# define the topic distributions 
d1 = CategoricalDistribution(pmap={"walk": 0.6, "shop": 0.3, "clean": 0.1})
d2 = CategoricalDistribution(pmap={"walk": 0.1, "shop": 0.4, "clean": 0.5})

# assume that we always observe 7 days in the week
len_dist = CategoricalDistribution(pmap={7: 1.0})
d = HiddenMarkovModelDistribution(topics=[d1, d2], transitions=tpm, w=w, len_dist=len_dist)

We generate `1000` sequences using the `sampler()` call on the distribution

In [8]:
data = d.sampler(seed=1).sample(1000)
print('\n'.join(['Sample %d: '%(i) + str(data[i]) for i in range(10)]))

Sample 0: ['clean', 'walk', 'walk', 'shop', 'shop', 'clean', 'walk']
Sample 1: ['shop', 'shop', 'clean', 'walk', 'walk', 'shop', 'walk']
Sample 2: ['walk', 'walk', 'shop', 'shop', 'shop', 'clean', 'walk']
Sample 3: ['clean', 'shop', 'clean', 'walk', 'clean', 'walk', 'walk']
Sample 4: ['walk', 'walk', 'walk', 'walk', 'clean', 'clean', 'walk']
Sample 5: ['walk', 'shop', 'walk', 'clean', 'clean', 'clean', 'clean']
Sample 6: ['clean', 'shop', 'clean', 'shop', 'clean', 'clean', 'shop']
Sample 7: ['walk', 'walk', 'clean', 'shop', 'clean', 'clean', 'walk']
Sample 8: ['shop', 'walk', 'walk', 'walk', 'walk', 'shop', 'shop']
Sample 9: ['shop', 'clean', 'clean', 'clean', 'shop', 'walk', 'clean']


## The Viterbi Algorithm: Finding the Most Likely Hidden State Sequence

### What is the Viterbi Algorithm?

The **Viterbi algorithm** is a dynamic programming algorithm that finds the most probable sequence of hidden states given a sequence of observations. Unlike computing the full likelihood (which sums over all possible hidden state sequences), Viterbi finds the single best path.

**Mathematical Formulation:**

Given observations $X = (x_1, x_2, ..., x_T)$, we want to find:

$$Z^* = \arg\max_{Z} P(Z \mid X) = \arg\max_{Z} P(X, Z)$$

The algorithm uses dynamic programming with the recursion:

$$\delta_t(j) = \max_{i} [\delta_{t-1}(i) \cdot a_{ij}] \cdot b_j(x_t)$$

where:
- $\delta_t(j)$ is the maximum probability of being in state $j$ at time $t$ given the observations up to time $t$
- $a_{ij}$ is the transition probability from state $i$ to state $j$
- $b_j(x_t)$ is the emission probability of observing $x_t$ in state $j$

**Initialization:**
$$\delta_1(j) = \pi_j \cdot b_j(x_1)$$

**Termination:**
$$P(Z^*) = \max_j \delta_T(j)$$

### Why Use Viterbi?

- **Decoding**: Infer the most likely hidden state sequence (e.g., what was the weather each day?)
- **Efficient**: $O(N^2T)$ complexity where $N$ is number of states and $T$ is sequence length
- **Optimal**: Guaranteed to find the globally optimal state sequence
- **Applications**: Speech recognition, bioinformatics (gene finding), activity recognition

### Viterbi in the Weather/Activities Example

Let's apply Viterbi to our toy example. Suppose we observe:
$$X = (\text{walk}, \text{shop}, \text{clean})$$

The Viterbi algorithm will determine: "What's the most likely weather pattern that would produce these activities?". We can easily run the Viterbi algorithm with a call to `viterbi()`. 

In [9]:
int_to_hidden_state = {0: 'rainy', 1: 'sunny'}
v_int = [d.viterbi(u) for u in data[:5]]
v_str = [[int_to_hidden_state[w] for w in v] for v in v_int]

print('\n'.join(['Sample: ' + str(data[i]) + '\nViterbi: ' + str(v_str[i]) + '\n' for i in range(len(v_str))]))

Sample: ['clean', 'walk', 'walk', 'shop', 'shop', 'clean', 'walk']
Viterbi: ['sunny', 'rainy', 'rainy', 'rainy', 'rainy', 'sunny', 'rainy']

Sample: ['shop', 'shop', 'clean', 'walk', 'walk', 'shop', 'walk']
Viterbi: ['rainy', 'sunny', 'sunny', 'rainy', 'rainy', 'rainy', 'rainy']

Sample: ['walk', 'walk', 'shop', 'shop', 'shop', 'clean', 'walk']
Viterbi: ['rainy', 'rainy', 'rainy', 'rainy', 'rainy', 'sunny', 'rainy']

Sample: ['clean', 'shop', 'clean', 'walk', 'clean', 'walk', 'walk']
Viterbi: ['sunny', 'sunny', 'sunny', 'rainy', 'sunny', 'rainy', 'rainy']

Sample: ['walk', 'walk', 'walk', 'walk', 'clean', 'clean', 'walk']
Viterbi: ['rainy', 'rainy', 'rainy', 'rainy', 'sunny', 'sunny', 'rainy']



## Parameter Estimation: Learning HMMs from Data

### The Challenge of Parameter Estimation

In real-world applications, we typically don't know the true HMM parameters ($\pi$, $A$, $B$). Instead, we have:
- **Observed data**: Sequences of observations (e.g., daily activities)
- **Unknown parameters**: Hidden states, transition probabilities, and emission probabilities

**Goal**: Learn the model parameters that best explain the observed data.

First we are going to define the `HiddenMarkovEstimator`. 


**Key Parameters:**
- **`estimators`**: A list of estimators for the topics.
- **`len_estimator`**: A length estimator for the length distribution.

In [12]:
est0 = CategoricalEstimator()
est = HiddenMarkovEstimator(
    estimators=[est0]*2, 
    len_estimator=est0
)

We can use `optimize` to fit the HMM.

In [18]:
fit = optimize(
    data=data,
    estimator=est,
    rng=np.random.RandomState(10),
    init_p=0.10,
    max_its=1000,
    delta=1.0e-4,
    print_iter=100)

Iteration 100: ln[p_mat(Data|Model)]=-7.601640e+03, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=1.001307e-01
Iteration 200: ln[p_mat(Data|Model)]=-7.571739e+03, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=3.750003e-03
Iteration 300: ln[p_mat(Data|Model)]=-7.571414e+03, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=2.833165e-03
Iteration 400: ln[p_mat(Data|Model)]=-7.571164e+03, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=2.199573e-03
Iteration 300: ln[p_mat(Data|Model)]=-7.571414e+03, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=2.833165e-03
Iteration 400: ln[p_mat(Data|Model)]=-7.571164e+03, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=2.199573e-03
Iteration 500: ln[p_mat(Data|Model)]=-7.570971e+03, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=1.684310e-03
Iteration 600: ln[p_mat(Data|Model)]=-7.570825e+03, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=1.259340e-03
Iteration 500: ln[p_mat(Data|Model)]=-7.570971e+03, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevMode

Lets inspect the estimated initial states and transition matrix.

In [28]:
print('Transition Probability Matrix: \n %s:'%(repr(fit.transitions)))
print('\nInitial Weights: \n %s:'%(repr(fit.w)))

Transition Probability Matrix: 
 array([[0.59770099, 0.40229901],
       [0.19625044, 0.80374956]]):

Initial Weights: 
 array([0.28064907, 0.71935093]):


These look reasonable. Now we can examine the estimated topics (emission distributions). 

In [29]:
print('\n\n'.join(['Topic %i: '%(i) + repr(fit.topics[i]) for i in range(2)]))

Topic 0: CategoricalDistribution({'clean': 0.5293729757856719, 'shop': 0.40647816522832164, 'walk': 0.06414885898600642}, default_value=0.0, name=None, keys=None)

Topic 1: CategoricalDistribution({'clean': 0.1457312028273226, 'shop': 0.3083313690267165, 'walk': 0.5459374281459609}, default_value=0.0, name=None, keys=None)


All the sequences had length 7, so our length distribution should reflect that...

In [30]:
fit.len_dist

CategoricalDistribution({7: 1.0}, default_value=0.0, name=None, keys=None)

Obviously there is not going to be an alignment with meaning and the hidden states. Earlier we were able to examine the weather as hidden states since we knew what they were before hand.

## Generalizing the HMM
The toy example we started with was a simple two state HMM with text emissions. This is not very exciting and can likely be handled with user written code or a different package. 

Lets look at an example now where `dmx-learn` can really help out. As per the norm, we will consider heterogenous tuples of data. Lets consider a case where we jointly observe sequences of `tuple[float, str]`. We will also assume the sequences have varying sizes. 

In [37]:
# define the topics for a 3 state HMM. 
v = 0.1
d1 = CompositeDistribution(
    (GaussianDistribution(-20.0, 1.0), CategoricalDistribution({'a': 0.5 - v, 'b': 0.5 - v, 'c': v, 'd': v})))
d2 = CompositeDistribution(
    (GaussianDistribution(0.0, 1.0), CategoricalDistribution({'a': v, 'b': v, 'c': 0.5 - v, 'd': 0.5 - v})))
d3 = CompositeDistribution(
    (GaussianDistribution(20.0, 1.0), CategoricalDistribution({'a': v, 'b': 0.5 - v, 'c': 0.5 - v, 'd': v})))
topics = [d1, d2, d3]

# define the length distribution
len_dist = PoissonDistribution(lam=4.0)

# initial state distribution
w = [0.4, 0.4, 0.2]

# transition distribution
tpm = [[0.8, 0.1, 0.1], [0.1, 0.8, 0.1], [0.1, 0.1, 0.8]]

# define the Hidden Markov model
dist = HiddenMarkovModelDistribution(topics=topics, w=w, transitions=tpm, len_dist=len_dist)


Draw `1000` samples from the HMM.

In [ ]:
sampler = dist.sampler(seed=32)
data = sampler.sample(1000)

We can print out a few samples..

In [39]:
for i, x in enumerate(data[:5]):
    print('Sample %i: [' %(i) +', '.join(['(%0.3f, %s)'%(xx[0], xx[1]) for xx in x])+']')

Sample 0: [(20.500, b)]
Sample 1: [(18.335, c), (-19.435, b), (-18.976, b), (-19.862, a)]
Sample 2: [(-20.280, a), (-0.318, c), (0.145, d), (0.572, d), (-20.958, a)]
Sample 3: []
Sample 4: [(-18.712, d), (-20.612, b), (-20.871, b), (-21.165, a)]


You may have noticed that some of the samples are empty. `dmx-learn` can handle this case. It will simply be treated as a sequence of length 0. 

In [43]:
# define the estimator for the tuple of data
est0 = GaussianEstimator()
est1 = CategoricalEstimator()
est2 = CompositeEstimator([est0, est1])

# define the length estimator
len_est = PoissonEstimator() 

# define the estimator for the topics (3 state hmm)
topic_ests = [est2] * 3 
est = HiddenMarkovEstimator(estimators=topic_ests, len_estimator=est1)

In [44]:
fit = optimize(
    data=data,
    estimator=est,
    rng=np.random.RandomState(45),
    init_p=0.10,
    max_its=1000,
    delta=1.0e-4,
    print_iter=100)

Iteration 21: ln[p_mat(Data|Model)]=-1.567520e+04, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=0.000000e+00


Lets look at the estimated topics.

In [50]:
for i, comp in enumerate(fit.topics):
    print(": ".join([str(i), str(comp)]))

0: CompositeDistribution(dists=[GaussianDistribution(19.981996424401455, 1.0244243730846847, name=None, keys=None),CategoricalDistribution({'a': 0.10598626104023552, 'b': 0.38763493621197254, 'c': 0.4023552502453386, 'd': 0.10402355250245339}, default_value=0.0, name=None, keys=None)], name=None, keys=None)
1: CompositeDistribution(dists=[GaussianDistribution(0.010639974870002197, 1.0455403103304144, name=None, keys=None),CategoricalDistribution({'a': 0.1051939513477975, 'b': 0.09072978303747535, 'c': 0.4089414858645628, 'd': 0.39513477975016437}, default_value=0.0, name=None, keys=None)], name=None, keys=None)
2: CompositeDistribution(dists=[GaussianDistribution(-19.998408273214554, 0.9110252706064443, name=None, keys=None),CategoricalDistribution({'a': 0.39422431161853594, 'b': 0.3955674949630625, 'c': 0.10476830087306917, 'd': 0.10543989254533244}, default_value=0.0, name=None, keys=None)], name=None, keys=None)


We can also look at the estimated transition matrix.

In [52]:
print('True TPM:\n' + repr(dist.transitions) + '\nEstimated TPM:\n' + repr(fit.transitions))

True TPM:
array([[0.8, 0.1, 0.1],
       [0.1, 0.8, 0.1],
       [0.1, 0.1, 0.8]])
Estimated TPM:
array([[0.79483696, 0.10869565, 0.09646739],
       [0.11073826, 0.77768456, 0.11157718],
       [0.09236234, 0.10657194, 0.80106572]])


The initial weight estimates

In [54]:
print('True initial state distribution:\n' + repr(dist.w) + '\nEstimated initial state distribution:\n' + repr(fit.w))

True initial state distribution:
array([0.4, 0.4, 0.2])
Estimated initial state distribution:
array([0.20307692, 0.40410256, 0.39282051])


Note that in the above, the estimates are correct, however the ordering of the component and weights may be permuted. This happens in estimation, just be sure to do proper book keeping when needed. 

# Language Identification With Coupled HMMs

Now we have worked through a toy example and generalized to see how `dmx-learn` can model sequential dependencies of heterogenous data tuples. We also saw that `dmx-learn` can easily handle sequences of varying lengths.

We'll now turn our attention to a data example in which we build a set of models for idenfiying languages with HMMs. We will use text exerpts from the Iliad to train three HMMs on English, French, and Spanish words. What will make this a bit more interesting is that the HMMs will have the same emission distributions. This means that the differences between the languages must be encoded in the transition probabilities and initial state probabilities. 

We randomly pick spots in each text and extract a fixed length sequence of words, $\{w_i\}_{i=1}^m$. The language of the text is appended and we then estimate the distribution 
$$P(\{w_i\}_{i=1}^m | \text{language}) = \prod_{i = 1}^m P(w_i | \text{language}).$$

where our model over the sequences of words are an HMM with the same emission distributions. 

### Data Preparation

Load each text, switch tolower case, remove the liscense stuff, and split on non-word characters. 

In [ ]:
import re
import codecs
import os

fin = codecs.open(os.path.join(DATA_LOC, 'iliad', 'iliad_en.txt'), encoding='utf-8', mode='r')
temp_en = fin.read()
fin.close()

fin = codecs.open(os.path.join(DATA_LOC, 'iliad', 'iliad_fr.txt'), encoding='utf-8', mode='r')
temp_fr = fin.read()
fin.close()

fin = codecs.open(os.path.join(DATA_LOC, 'iliad', 'iliad_es.txt'), encoding='utf-8', mode='r')
temp_es = fin.read()
fin.close()

end_en = 'end of the project gutenberg ebook'
end_fr = 'end of the project gutenberg ebook'
end_es = 'end of the project gutenberg ebook'

start_en = 'sing, goddess, the wrath of achilles'
start_fr = 'chante, déesse, du pèlèiade akhilleus'
start_es = 'canta, oh diosa, la cólera del pelida aquiles'

temp_en = temp_en.lower().split(end_en)[0]
temp_fr = temp_fr.lower().split(end_fr)[0]
temp_es = temp_es.lower().split(end_es)[0]

temp_en = start_en + temp_en.split(start_en, 1)[1]
temp_fr = start_fr + temp_fr.split(start_fr, 1)[1]
temp_es = start_es + temp_es.split(start_es, 1)[1]

# Use raw string for regex
temp_en = list(filter(lambda u: len(u) > 0, re.split(r'\W+', temp_en)))
temp_fr = list(filter(lambda u: len(u) > 0, re.split(r'\W+', temp_fr)))
temp_es = list(filter(lambda u: len(u) > 0, re.split(r'\W+', temp_es)))


For each language we will randomly pick spots throughout the text and extract text sequences. The true langauge is appended to make the data look like `(language ID, {word, word, ...})`. These are then split into a training (90% of data) and testing set (10% of data).

In [61]:
rng = np.random.RandomState(1)
seq_len = 5
num_seq = 1000
split_idx = int(num_seq*0.9)

idx_en = rng.choice(int(len(temp_en)/seq_len), num_seq, replace=False)
idx_fr = rng.choice(int(len(temp_fr)/seq_len), num_seq, replace=False)
idx_es = rng.choice(int(len(temp_es)/seq_len), num_seq, replace=False)

data_en = [(0, temp_en[(i*seq_len):((i+1)*seq_len)]) for i in idx_en]
data_fr = [(1, temp_fr[(i*seq_len):((i+1)*seq_len)]) for i in idx_fr]
data_es = [(2, temp_es[(i*seq_len):((i+1)*seq_len)]) for i in idx_es]

train_data = data_en[:split_idx] + data_fr[:split_idx] + data_es[:split_idx]
test_data  = data_en[split_idx:] + data_fr[split_idx:] + data_es[split_idx:]

Here are some samples from each language.

In [62]:
print('\n'.join(map(str,data_en[0:5])))
print('\n'.join(map(str,data_fr[0:5])))
print('\n'.join(map(str,data_es[0:5])))

(0, ['pour', 'their', 'wine', 'then', 'would'])
(0, ['he', 'saw', 'lying', 'on', 'the'])
(0, ['strife', 'and', 'wars', 'and', 'battles'])
(0, ['not', 'even', 'one', 'from', 'among'])
(0, ['thy', 'dart', 'at', 'this', 'fellow'])
(1, ['ont', 'gardé', 'ce', 'qu', 'il'])
(1, ['mains', 'vigoureuses', 'un', 'créneau', 'du'])
(1, ['même', 'char', 'vit', 'au', 'loin'])
(1, ['par', 'serment', 'aux', 'troiens', 'de'])
(1, ['dans', 'l', 'olympos', 'qui', 'est'])
(2, ['construído', 'trípode', 'pero', 'ni', 'ulises'])
(2, ['dentro', 'ya', 'del', 'magnífico', 'palacio'])
(2, ['xxii', '46', 'á', '48', '2'])
(2, ['obstina', 'en', 'rechazarlas', 'se', 'dirigen'])
(2, ['y', 'dijo', 'el', 'eximio', 'vate'])


#### Estimation

We are going to build an initial estimator and an estimator for fitting the model. First we will need to consider our modeling objective. The model we will estimate is a conditional distribution with three given integer values 0 (English), 1 (Greek), 2 (Spanish) for the languages. The text sequences will be modeled as a sequence of 20 state HMMs with categorical emissions that are shared among each language we have conditioned on. 

We want to build a model for three languages, where each model is an HMM with the same topics. Our data looks like `tuple[int, list[str]]`. We will use a `ConditionalDistributionEstimator` to handle the different language types, `SequenceEstimator` to handle the sequence of words, and an `HiddenMarkovEstimator` words. Since the emissions of the HMM are strings (letters of the words), we will use a `CategoricalEstimator` for the emission distributions. 

To ensure that our conditional models share topics for the HMMs, we will need to key the `HiddenMarkovEstimator`. We will also need to flatten our topic distributions to ensure that each letter has positive probability to avoid numerical underflow. These two constraints are met by passing `keys` to the `HiddenMarkovEstimator` and setting a `suff_stat` for the topics. These variables are described by:

1. `keys = (None, None, 'topics')`: This will be used to ensure that the emission distributions of the HMM are the same.

2. `suff_stat`: This is a dictionary mapping the words of all languages to a positive float. This is merely a method to ensure that each emission distribution has a non-zero probability for each word in the training set. 


In [97]:
# This will ensure sure that every letter in the training set has non-zero probability
all_langs = list(sorted(list(set([u[0] for u in train_data]))))
suff_stat = count_by_value([w for u in train_data for v in u[1] for w in v])
tcnt = sum(suff_stat.values())
suff_stat = {k:v/tcnt for k,v in suff_stat.items()}

num_states = 20
# define the flattened initial estimator for words
iest0 = CategoricalEstimator(pseudo_count=1.0, suff_stat=suff_stat)

# define the HMM and initial estimator
iest1 = HiddenMarkovEstimator(estimators=[iest0] * num_states, pseudo_count=(1.0, 1.0), keys=(None, None, "topics"), use_numba=True)
iest2 = SequenceEstimator(iest1)
iest = ConditionalDistributionEstimator(estimator_map={u: iest2 for u in all_langs})

# define the flattened estimator for words
est0 = CategoricalEstimator(pseudo_count=1.0e-6, suff_stat=suff_stat)

# define the estimator
est1 = HiddenMarkovEstimator(estimators=[est0] * num_states, pseudo_count=(1.0e-6, 1.0e-6), keys=(None, None, "topics"), use_numba=True)
est2 = SequenceEstimator(est1)
est = ConditionalDistributionEstimator(estimator_map={u: est2 for u in all_langs})

In [98]:
fit = optimize(
    data=train_data,
    estimator=est,
    rng=np.random.RandomState(53),
    init_p=1.0,
    init_estimator=iest,
    max_its=1000,
    delta=1.0e-4,
    print_iter=250)

Iteration 250: ln[p_mat(Data|Model)]=-1.401184e+05, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=3.596256e+00
Iteration 500: ln[p_mat(Data|Model)]=-1.398345e+05, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=1.362659e+00
Iteration 500: ln[p_mat(Data|Model)]=-1.398345e+05, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=1.362659e+00
Iteration 750: ln[p_mat(Data|Model)]=-1.395108e+05, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=4.127860e-01
Iteration 750: ln[p_mat(Data|Model)]=-1.395108e+05, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=4.127860e-01
Iteration 1000: ln[p_mat(Data|Model)]=-1.394234e+05, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=5.880658e-02
Iteration 1000: ln[p_mat(Data|Model)]=-1.394234e+05, ln[p_mat(Data|Model)]-ln[p_mat(Data|PrevModel)]=5.880658e-02


The function `get_classification_matrix` will compute the likelihood of each sample given each language and then select the langauge with the highest likelhood. A matrix of counts, where the row is the true class and the column is the predict class, is returned. The function `pp` will make a fixed precision string for given matrix.

In [99]:
def get_classification_matrix(model, data):
    
    # Extract the labels from the model
    cvals = list(sorted(list(model.dmap.keys())))
    cvals_cnt = len(cvals)
    cvals_map = dict(zip(cvals, range(cvals_cnt)))
    
    res = np.zeros((cvals_cnt, cvals_cnt))
    ll_mat = np.zeros((cvals_cnt, len(data)))
    
    # Make a list of the true labels
    true_idx_list = [cvals_map[u[0]] for u in data]
    
    for i in range(cvals_cnt):
        # Replace the true label with the test label
        ll_mat[i,:] = model.seq_log_density(model.dist_to_encoder().seq_encode([(cvals[i], u[1]) for u in data]))
        
    # Find the most likely language
    test_idx_list = np.argmax(ll_mat, axis=0)
    
    # Enumerate the entries and count the true and predicted languages
    for true_idx, test_idx in zip(true_idx_list, test_idx_list):
        res[true_idx, test_idx] += 1

    return res

def pp(x):
    return '\n'.join([','.join(['%0.3f'%(v) for v in x[i,:]]) for i in range(x.shape[0])])

The results should be pretty good (somewhere around the high 90s) when the classifier is applied to the training set and the testing set.

In [102]:
train_res = get_classification_matrix(fit,train_data)
print('Train')
print(pp(train_res / np.sum(train_res, axis=1, keepdims=True)))

test_res = get_classification_matrix(fit,test_data)
print('Test')
print(pp(test_res / np.sum(test_res, axis=1, keepdims=True)))

Train
0.984,0.004,0.011
0.003,0.972,0.024
0.002,0.016,0.982
Test
0.960,0.020,0.020
0.000,0.950,0.050
0.010,0.030,0.960


So there you have it, we can learn the differences in the languages quite well simply through the transition densities and intial state vectors. 

This notebook tutorial should serve as a primer on HMMs in `dmx-learn`. We also saw how flattening and key'ing distributions can come in handy for defining models with shared parameters.